In [ ]:
# dg test ir4 chips  (based on casting void - cast03 e03cast.ipynb)

Next, change directory to wherever you created your folder. 

In [ ]:
import os

# TODO: Change this to your Drive folder location
WORKING_DIRECTORY = '/notebooks/ml639a/639e'
imgdir='/notebooks/imgdata/inrm4pyt'
imgtrain = '/notebooks/imgdata/inrm4pyt/train'
imgval = '/notebooks/imgdata/inrm4pyt/val'

os.chdir(WORKING_DIRECTORY)
!pwd

In [ ]:
# List the contents of your working directory
# It should contain at least three folders: images, train_labels, and val_labels
!ls -la

Now, let's install the Detecto package using pip. 

In [ ]:
# Note: if it states you must restart the runtime in order to use a
# newly installed version of a package, you do NOT need to do this. 
!pip install detecto

Import everything we need in the following code block:

In [ ]:
!apt-get install -y python3-opencv

In [ ]:
pip install opencv-python

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torchvision import transforms
from detecto import core, utils, visualize

To check that everything's working, we can try reading in one of the images from our images folder. 

In [ ]:
%%bash
find ./images -type f | tail -n1

In [ ]:
image = utils.read_image('/notebooks/imgdata/inrm4pyt/train/inner_rim_210805T103940.png')
plt.imshow(image)
plt.show()

How cute! Now, we're ready to create our dataset and train our model. However, before doing so, it's a bit slow working with hundreds of individual XML label files, so we should convert them into a single CSV file to save time later down the line. 

In [ ]:
# Do this twice: once for our trning labels and once for our validation labels
utils.xml_to_csv(imgtrain, 'train.csv')
utils.xml_to_csv(imgval, 'val.csv')

Below, we create our dataset, applying a couple of transforms beforehand. These are optional, but they can be useful for augmenting your dataset without gathering more data. 

In [29]:
# Specify a list of transformations for our dataset to apply on our images
transform_img00 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(800),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    utils.normalize_transform(),
])

transform_img = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([512,256]),
    transforms.ToTensor(),
    utils.normalize_transform(),
])

dataset = core.Dataset('train.csv', imgtrain, transform=transform_img)

# dataset[i] returns a tuple containing our transformed image and
# and a dictionary containing label and box data
image, target = dataset[0]

# Show our image along with the box. Note: it may
# be colored oddly due to being normalized by the 
# dataset and then reverse-normalized for plotting
visualize.show_labeled_image(image, target['boxes'], target['labels'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Finally, let's train our model! First, we create a DataLoader over our dataset to specify how we feed the images into our model. We also use our validation dataset to track the accuracy of the model throughout training. 

In [19]:
!date -d "-4 hours"

Wed Jun 29 20:45:12 UTC 2022


In [23]:
# Create our validation dataset
!date -d "-4 hours"
val_dataset = core.Dataset('val.csv', imgval)

# Create the loader for our training dataset
loader = core.DataLoader(dataset, batch_size=1, shuffle=True)

# Create our model, passing in all unique classes we're predicting
# Note: make sure these match exactly with the labels in the XML/CSV files!
model = core.Model(['Chip'])

# Train the model! This step can take a while, so make sure you
# the GPU is turned on in Edit -> Notebook settings
losses = model.fit(loader, val_dataset, epochs=3, verbose=True)

# Plot the accuracy over time
plt.plot(losses)
plt.show()
!date -d "-4 hours"


Wed Jun 29 20:46:32 UTC 2022
Epoch 1 of 3
Begin iterating over training dataset


  0%|          | 0/4 [00:00<?, ?it/s]


AssertionError: 

Let's see how well our model does on a couple images from our validation set:

In [ ]:
images = []
# Create a list of images  from val_dataset
for i in range(5, 9, 1):
    image, _ = val_dataset[i]
    images.append(image)
    print(image)

# Plot a 3x3 grid of the model's predictions on our 9 images
visualize.plot_prediction_grid(model, images, dim=(2, 2), figsize=(28, 24))

In [ ]:
!find ./testfinal -type f | tail -n -3
!pwd

In [ ]:
f1=os.listdir("./testfinal")
print(f1)
import pathlib
for filepath in pathlib.Path("testfinal/").glob('**/*'):
    print(filepath.absolute())

In [ ]:
# one image prediction.

image = utils.read_image('./testfinal/cast_def_0_9591.jpeg')
predictions = model.predict(image)

images = []
# for i in range(4):
#     image, _ = val_dataset[i]
images.append(image)

top_predictions = model.predict_top(images)

print(predictions)
print(top_predictions)
visualize.plot_prediction_grid(model, images, dim=(1, 1), figsize=(28, 28))


In [ ]:
# error -  IndexError: index 0 is out of bounds for axis 0 with size 0

!pwd
testfinal_data = core.Dataset( 'testfinal/')
print(val_dataset)
print(val_dataset[0])
print(testfinal_data)
print(testfinal_data[1])
images = []
for i in range(0,3,1):
    print(i)
    image, _ = testfinal_data[i]
    images.append(image)
top_predictions = model.predict_top(images)
# print(predictions)
print(top_predictions)    
visualize.plot_prediction_grid(model, images, dim=(4, 1), figsize=(128, 128))

In [ ]:
# 203 loop show predictions

images = []
import pathlib
for filepath in pathlib.Path("testfinal/").glob('**/*'):
    print(filepath.absolute())
    path01=str(filepath.absolute())
    image = utils.read_image(path01)
    images.append(image)

# top_predictions = model.predict_top(images)
# print(top_predictions)

visualize.plot_prediction_grid(model, images, dim=(4, 1), figsize=(44, 44))


In [ ]:
import datetime
dtnow = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
fout = f"../../modelweights_cast03_{dtnow}.pth"
model.save(fout)
# model.save("modelweight.pth")
!pwd;ls

## Conclusion

Thanks for making it this far through the demo!

This is as far as the demo goes, but a great next step would be seeing how well the model works on a live video of Chihuahuas and Golden Retrievers in the same frame at the same time. To learn more about Detecto, be sure to check out the [Quickstart guide](https://detecto.readthedocs.io/en/latest/usage/quickstart.html), [Further Usage guide](https://detecto.readthedocs.io/en/latest/usage/further-usage.html), and [API docs](https://detecto.readthedocs.io/en/latest/api.html)!